Following the TensorFlow Tutorial #20 from GitHub

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
!pip install numpy==1.16.1
import numpy as np
from scipy.spatial.distance import cdist

from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding
from keras.optimizers import Adam
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [0]:
#from tensorflow.python.keras.layers import Dense, GRU, Embedding
#from tensorflow.python.keras.optimizers import Adam
#from tensorflow.python.keras.preprocessing.text import Tokenizer

In [0]:
tf.__version__

In [0]:
tf.keras.__version__

Install once per run, for import imdb to work

In [0]:
!pip install imdbpy

In [0]:
import imdb

In [0]:
imdbO = imdb.IMDb()

object_methods = [method_name for method_name in dir(imdbO)
                  if callable(getattr(imdbO, method_name))]
print(object_methods)

In [0]:
from keras.datasets import imdb
# load the dataset
(X_train, y_train), (X_test, y_test) = imdb.load_data()


In [0]:
print("X_train[0]", X_train[0])
print("y_train[0]", y_train[0])
print("X_test[0]", X_test[0])
print("y_test[0]", y_test[0])

In [0]:
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)
print("X[0]", X[0])
print("y[0]", y[0])

In [0]:
# summarize size
print("Training data: ")
print(X.shape)
print(y.shape)

In [0]:
# Summarize number of classes
print("Classes: ")
print(np.unique(y))

In [0]:
# Summarize number of words
print("Number of words: ")
print(len(np.unique(np.hstack(X))))

In [0]:
# Summarize review length
print("Review length: ")
result = [len(x) for x in X]
print("Mean %.2f words (%f)" % (np.mean(result), np.std(result)))
# plot review length
plt.boxplot(result)
plt.show()

# **Preparing the data**

In [0]:
# User defined parameters
batch_size_user = 128
embedding_size = 32
num_words = 5000
max_tokens = 500
epoch_user = 2

In [0]:
# load the dataset but only keep the top n words, zero the rest
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

In [0]:
X_train = sequence.pad_sequences(X_train, maxlen=max_tokens)
X_test = sequence.pad_sequences(X_test, maxlen=max_tokens)

# **Preparing the models for anaylsis**

In [0]:
# create the model
model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_tokens))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
flatten_3 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 250)               4000250   
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 251       
Total params: 4,160,501
Trainable params: 4,160,501
Non-trainable params: 0
_________________________________________________________________
None


Batch size affects the number of samples exposed to the network per run before updating it, more samples would take more memory.


Increasing batch size trains the network faster as lesser updating is done which leads to lesser accuracy.

Conversely, decreasing batch size trains the network slower, but higher accuracy is achieved as there is more updating sessions with the same number of epochs. However risk overfitting.

In [0]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epoch_user, batch_size=batch_size_user, verbose=1)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 32s 1ms/step - loss: 0.5084 - acc: 0.7087 - val_loss: 0.3318 - val_acc: 0.8552
Epoch 2/2
25000/25000 [==============================] - 30s 1ms/step - loss: 0.1887 - acc: 0.9292 - val_loss: 0.3033 - val_acc: 0.8720
Accuracy: 87.20%


2nd Model attempt, with RNN/GRU included before the NN portion

In [0]:
model2 = Sequential()



model2.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))
model2.add(GRU(units=16, return_sequences=True))
model2.add(GRU(units=8, return_sequences=True))
model2.add(GRU(units=4))
model2.add(Dense(250, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
optimizer = Adam(lr=1e-3)
model2.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model2.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 500, 32)           160000    
_________________________________________________________________
gru_4 (GRU)                  (None, 500, 16)           2352      
_________________________________________________________________
gru_5 (GRU)                  (None, 500, 8)            600       
_________________________________________________________________
gru_6 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_11 (Dense)             (None, 250)               1250      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 251       
Total params: 164,609
Trainable params: 164,609
Non-trainable params: 0
_________________________________________________________________


In [0]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epoch_user, batch_size=batch_size_user, verbose = 1)

# Final evaluation of the model
scores = model2.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 149s 6ms/step - loss: 0.5432 - acc: 0.7063 - val_loss: 0.3635 - val_acc: 0.8483
Epoch 2/2
25000/25000 [==============================] - 152s 6ms/step - loss: 0.2988 - acc: 0.8792 - val_loss: 0.3262 - val_acc: 0.8668
Accuracy: 86.68%


When evaluating the 2nd model with the added RNN/GRU before the Dense network, the accuracy climb noticably has backward drops regularly as compared to the 1st model.

Additional, a longer duration is required to train the 2nd model with RNN/GRU

In [0]:
# create the model
model3 = Sequential()
model3.add(Embedding(num_words, embedding_size, input_length=max_tokens))
model3.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Flatten())
model3.add(Dense(250, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 250)               2000250   
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 251       
Total params: 2,163,605
Trainable params: 2,163,605
Non-trainable params: 0
_________________________________________________________________


In [0]:
model3.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epoch_user, batch_size=batch_size_user, verbose = 1)

# Final evaluation of the model
scores = model3.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 33s 1ms/step - loss: 0.5014 - acc: 0.7188 - val_loss: 0.2820 - val_acc: 0.8830
Epoch 2/2
25000/25000 [==============================] - 32s 1ms/step - loss: 0.2197 - acc: 0.9142 - val_loss: 0.2689 - val_acc: 0.8879
Accuracy: 88.79%
